In [ ]:
import nibabel as nib
import numpy as np
import glob

import itertools
from sklearn.metrics import jaccard_score, f1_score

In [ ]:
batch_root = "/Users/nicklittlefield/OneDrive - University of Pittsburgh/Dataset_Knee_Bony_Anatomy_Segmentation(Lead_Sakshi)/Batch#2 (same images)_10_Images_for_Level_of_Agreement/"
annotator_dirs = list(glob.iglob(batch_root + "*"))
num_annotators = len(annotator_dirs)
num_annotators

In [ ]:
print("Found Annotator Directories:")
for annotator in annotator_dirs:
    print(annotator)

In [ ]:
# Get common files among the annotators
import regex as re
pid_regex = re.compile("9[0-9]{6}")
file_ids = set([pid_regex.findall(file)[0] for file in sorted(list(glob.iglob(annotator_dirs[0] + "/*.nii.gz")))])
for annotator in annotator_dirs[1:]:
    annotator_files = [pid_regex.findall(file)[0] for file in sorted(list(glob.iglob(annotator + "/*.nii.gz")))]
    file_ids = file_ids.intersection(annotator_files)


In [ ]:
file_ids

In [ ]:
iou_scores= []
dice_scores = []
annot_pairs = list(itertools.combinations(annotator_dirs, 2))

for file in file_ids:
    annot_pair_ious = []
    annot_pair_dice = []
    
    for (annot1, annot2) in annot_pairs:
        annot_mask1_pth = f"{annot1}/{file}.nii.gz"
        annot_mask2_pth = f"{annot2}/{file}.nii.gz"
        
        # Load Masks
        annot_mask1 = nib.load(annot_mask1_pth).get_fdata()
        annot_mask2 = nib.load(annot_mask2_pth).get_fdata()
        
        # Check to make sure number of channels = 1, if not convert to one channel (middle)
        if annot_mask1.shape[2] > 1:
            annot_mask1 = annot_mask1[:, :, annot_mask1.shape[2]//2]
        
        if annot_mask2.shape[2] > 1:
            annot_mask2 = annot_mask2[:, :, annot_mask2.shape[2]//2]
    
        iou = jaccard_score(annot_mask1.flatten(), annot_mask2.flatten(), average="macro")
        dice = f1_score(annot_mask1.flatten(), annot_mask2.flatten(), average="macro")
        
        annot_pair_ious.append(iou)
        annot_pair_dice.append(dice)
    
    iou_scores.append(np.mean(annot_pair_ious))
    dice_scores.append(np.mean(annot_pair_dice))


In [ ]:
np.mean(iou_scores), np.mean(dice_scores)